In [1]:
import nengo
import nengo_spa as spa
import matplotlib.pyplot as plt
%matplotlib inline
# with and without seed
import numpy as np
import random
import numpy as np
import pickle
d = 128  # the dimensionality of the vectors
# note dimension
SDT_variable = 3
proportion_ablate = 0.0
noise_std = 0.005
time_interval = 1.0
wait =  0.0
feedback_synapse = 0.1

with spa.Network() as model:
        
       
        feedback = 1        
 
            
        noise = nengo.Node(nengo.processes.WhiteNoise(nengo.dists.Gaussian (0,noise_std)), size_out = d)
        timer = time_interval + wait
        vocab = spa.Vocabulary(d)
        vocab.populate('BLUE; RED; GREEN')
        GREEN = vocab.parse('GREEN')
        RED = vocab.parse('RED')
        BLUE = vocab.parse('BLUE')
            
       
        Color = spa.State(vocab, feedback = feedback, represent_cc_identity = False)
        Color_1 = spa.State(vocab, feedback = feedback, represent_cc_identity = False)
        Color_2 = spa.State(vocab, feedback = feedback, represent_cc_identity = False)
        
        nengo.Connection(noise,Color.input, synapse = 0.01)
        
        Color >> Color_1
        Color_1 >> Color_2
        
        def input(t):
                if t < time_interval:
                    return BLUE
                elif t < timer:
                    return '0'
                elif t < timer + time_interval:
                    return RED
                elif t < 2 * timer:
                    return '0'
                elif t < 2 * timer + time_interval:
                    return GREEN
                elif t < 3 * timer:
                    return '0'
                else:
                    return '0'
        
        
        color_trans = spa.Transcode(function = input, output_vocab = vocab)
        color_trans >> Color
        probe_Color = nengo.Probe(Color.output, synapse = 0.01) 
        probe_Color_1 = nengo.Probe(Color_1.output, synapse = 0.01) 
        probe_Color_2 = nengo.Probe(Color_2.output, synapse = 0.01) 
        
        def ablate_ensemble(ens, proportion_ablate, sim, bias=True):
                n_neurons = min(int(ens.n_neurons * proportion_ablate), ens.n_neurons)
                idx = np.random.choice(np.arange(ens.n_neurons), replace=False, size=n_neurons)
                encoder_sig = sim.signals[sim.model.sig[ens]['encoders']]
                encoder_sig.setflags(write=True)
                encoder_sig[idx] = 0.0
                encoder_sig.setflags(write=False)
                if bias:

                    bias_sig = sim.signals[sim.model.sig[ens.neurons]['bias']]
                    bias_sig.setflags(write=True)
                    bias_sig[idx] = -1000



        
        with nengo.Simulator(model) as sim:  # Create the simulator
                for j in range (0, int(d/16)):
                    ens = Color.all_ensembles[j]
                    ablate_ensemble(ens, proportion_ablate, sim )
               
                sim.run(timer * 2.0)  
numbers  = spa.similarity(sim.data[probe_Color], vocab, normalize = True)
numbers_1  = spa.similarity(sim.data[probe_Color_1], vocab, normalize = True)                
numbers_2  = spa.similarity(sim.data[probe_Color_2], vocab, normalize = True)
                
#numbers_2_array = np.asarray(numbers_2)   

pickle.dump(numbers, open("howiesingle", "wb"))
sallysingle = pickle.load(open("howiesingle", "rb"))

pickle.dump(numbers_1, open("howiedouble", "wb"))
sallydouble = pickle.load(open("howiedouble", "rb"))


pickle.dump(numbers_2, open("howietriple", "wb"))
sallytriple = pickle.load(open("howietriple", "rb"))
x_values = np.linspace(0,2.0, 2000)   
        
plt.figure()
        
xmin, xmax = 0.9, 1.5
ymin, ymax = -0.2, 1.5
plt.grid(True)   
plt.title("Figure 9")
plt.xlabel("Time (s)")
plt.ylabel("Vector representation")
plt.ylim(ymin, ymax)
plt.xlim(xmin, xmax)

plt.plot(x_values,sallysingle[:,0], 'b-', label = "blue single", linewidth = 0.5)
plt.plot(x_values,sallydouble[:,0], 'b--', label = "blue double")
plt.plot(x_values,sallytriple[:,0], 'b:', label = "blue triple")
        
plt.plot(x_values,sallysingle[:,1], 'r-', label = "red single", linewidth = 0.5)
plt.plot(x_values,sallydouble[:,1], 'r--', label = "red double")
plt.plot(x_values,sallytriple[:,1], 'r:', label = "red triple") 

#plt.legend(['blue single' ,'blue double', 'blue triple', 'red single', 'red double', 'red triple' ],  loc='best')
#plt.tight_layout()
plt.savefig('SerialProcessingaug4.pdf')
                                                       
bb=np.array([1])
plt.plot(bb * 1.13, bb * 0.67,marker="H", fillstyle='none', markersize=20, linewidth=0)
plt.plot(bb * 1.23,bb * 0.65,marker="H", fillstyle='none', markersize=20, linewidth=0)
plt.plot(bb * 1.37,bb * 0.67,marker="H", fillstyle='none', markersize=20, linewidth=0)        
        
plt.savefig('Aug4_serial_proc'+ '.png')

IndentationError: unexpected indent (<ipython-input-1-272764c2ef5c>, line 133)